해당 노드는 다음과 같이 구성됩니다.
1. Data import & preprocessing
2. EDA
3. My preference
4. CSR Matrix
5. Training
6. recommendation

# 1) 데이터 전처리

In [1]:
import os
import pandas as pd
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [4]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [5]:
ratings.head()

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding = "ISO-8859-1")
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
movies['title'] = movies['title'].str.lower()
movies.head()

,movie_id,title,genre
0,1,toy story (1995),Animation|Children's|Comedy
1,2,jumanji (1995),Adventure|Children's|Fantasy
2,3,grumpier old men (1995),Comedy|Romance
3,4,waiting to exhale (1995),Comedy|Drama
4,5,father of the bride part ii (1995),Comedy


In [8]:
movies.loc[1:1, :]

,movie_id,title,genre
1,2,jumanji (1995),Adventure|Children's|Fantasy


In [9]:
result = []
for i in movies.loc[:, 'title']:
    result.append(i[:-7])
# print(result)
# print(result)

In [10]:
# 리메이크, 혹은 같은 이름으로 된 영화가 있는지 확인
# 년도를 제거했기 때문에 중복값이 생겼는지 확인
print(movies['title'].unique().shape)
len(set(result))

(3883,)


3841

In [11]:
# 이런식으로 변경하려 했는데 중복값이 있어서 안됨
# movies['title'] = result

In [12]:
movies.head()

,movie_id,title,genre
0,1,toy story (1995),Animation|Children's|Comedy
1,2,jumanji (1995),Adventure|Children's|Fantasy
2,3,grumpier old men (1995),Comedy|Romance
3,4,waiting to exhale (1995),Comedy|Drama
4,5,father of the bride part ii (1995),Comedy


In [13]:
print(movies['movie_id'][0])

1


# 2) 분석해 봅시다.
 * ratings에 있는 유니크한 영화 개수
 * rating에 있는 유니크한 사용자 수
 * 가장 인기 있는 영화 30개(인기순)

In [14]:
ratings['movie_id'].nunique()

3628

In [15]:
ratings['user_id'].nunique()

6039

In [16]:
user_num = ratings.groupby('movie_id')['count'].count()
user_num.head()

movie_id
1    2000
2     551
3     339
4     102
5     214
Name: count, dtype: int64

In [17]:
user_num.sort_values(ascending = False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: count, dtype: int64

In [18]:
movie_rate = ratings.groupby('user_id')['movie_id'].count()
movie_rate.head()

user_id
1     53
2    116
3     46
4     19
5    143
Name: movie_id, dtype: int64

In [19]:
# 영화별 몇명의 관객이 봤는지에 대한 통계
user_count = ratings.groupby('movie_id')['count'].count()
print(user_count.head())
user_count.describe()

movie_id
1    2000
2     551
3     339
4     102
5     214
Name: count, dtype: int64


count    3628.000000
mean      230.561742
std       355.596393
min         1.000000
25%        23.000000
50%        87.000000
75%       285.000000
max      3211.000000
Name: count, dtype: float64

In [20]:
# 관람객별 몇개의 영화를 봤는지에 대한 통계
user_count = ratings.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

# 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.

In [21]:
movies['title'].nunique()

3883

In [22]:
(movies.loc[:,'title'])

0                         toy story (1995)
1                           jumanji (1995)
2                  grumpier old men (1995)
3                 waiting to exhale (1995)
4       father of the bride part ii (1995)
                       ...                
3878               meet the parents (2000)
3879            requiem for a dream (2000)
3880                      tigerland (2000)
3881               two family house (2000)
3882                 contender, the (2000)
Name: title, Length: 3883, dtype: object

In [23]:
# 초기 추천을 위한 선호영화
my_favorite = ['toy story (1995)' , 'jumanji (1995)' ,'ghost and the darkness, the (1996)' ,'forrest gump (1994)' ,'bug\'s life, a (1998)']

In [24]:
# 영화 제목을 id로 바꿔줌
movie_idx = []
for i, k in enumerate(movies['title']):
    if k in my_favorite:
        print(k)
        print(movies['movie_id'][i])
        movie_idx.append(movies['movie_id'][i])

toy story (1995)
1
jumanji (1995)
2
forrest gump (1994)
356
ghost and the darkness, the (1996)
1049
bug's life, a (1998)
2355


In [25]:
print(movie_idx)

[1, 2, 356, 1049, 2355]


In [26]:
# 'zimin'이라는 user_id가 영화를 5번씩 봤다고 가정
my_playlist = pd.DataFrame({'user_id': ['zimin']*5, 'movie_id': movie_idx, 'count':[5]*5})

if not ratings.isin({'user_id':['zimin']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    ratings = ratings.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,count,timestamp
1000203,6040,1090,3,956715518.0
1000205,6040,1094,5,956704887.0
1000206,6040,562,5,956704746.0
1000207,6040,1096,4,956715648.0
1000208,6040,1097,4,956715569.0
0,zimin,1,5,NaN
1,zimin,2,5,NaN
2,zimin,356,5,NaN
3,zimin,1049,5,NaN
4,zimin,2355,5,NaN


In [27]:
ratings = ratings.drop(columns = 'timestamp')
ratings

,user_id,movie_id,count
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
0,zimin,1,5
1,zimin,2,5
2,zimin,356,5
3,zimin,1049,5


### 모델에 활용하기 위한 전처리(indexing)

In [28]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

# 유저, movie_id indexing
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [29]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['zimin'])    # 6039명의 유저 중 마지막으로 추가된 유저 
print(movie_to_idx[1090])

6039
1030


In [30]:
ratings['user_id'].map(user_to_idx.get).dropna()

0       0
1       0
2       0
3       0
4       0
     ... 
0    6039
1    6039
2    6039
3    6039
4    6039
Name: user_id, Length: 836483, dtype: int64

In [31]:
ratings['movie_id'].map(movie_to_idx.get).dropna()

0      0
1      1
2      2
3      3
4      4
    ... 
0     40
1    513
2    160
3    176
4      4
Name: movie_id, Length: 836483, dtype: int64

In [32]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('artist column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('artist column indexing Fail!!')

ratings

user_id column indexing OK!!
artist column indexing OK!!


,user_id,movie_id,count
0,0,0,5
1,0,1,3
2,0,2,3
3,0,3,4
4,0,4,5
...,...,...,...
0,6039,40,5
1,6039,513,5
2,6039,160,5
3,6039,176,5


In [63]:
# 영화 시청 횟수에 따른 비율
under_2 = ratings[ratings['count']<3]
only_3 = ratings[ratings['count']==3]
only_4 = ratings[ratings['count']==4]
only_5 = ratings[ratings['count']==5]

under_2 = len(under_2)
three = len(only_3)
four = len(only_4)
five = len(only_5)
all_data = len(ratings)


print(f'{all_data}')
print(f'Ratio of under_2 over all data is {under_2/all_data:.2%}')
print(f'Ratio of only_3 over all data is {three/all_data:.2%}')
print(f'Ratio of only_4 over all data is {four/all_data:.2%}')
print(f'Ratio of only_5 over all data is {five/all_data:.2%}')

836483
Ratio of under_2 over all data is 0.00%
Ratio of only_3 over all data is 31.23%
Ratio of only_4 over all data is 41.72%
Ratio of only_5 over all data is 27.06%


영화의 별점을  본 횟수라고 정의했을 때, 한사람이 2회 이하 본 비율은 0%이고, 3회 본 비율은 31.23%, 4회 본 비율은 41.72%, 5회 본 비율은 27.05%로 나타났습니다.

In [34]:
ratings['count']

0    5
1    3
2    3
3    4
4    5
    ..
0    5
1    5
2    5
3    5
4    5
Name: count, Length: 836483, dtype: int64

In [35]:
ratings.user_id

0       0
1       0
2       0
3       0
4       0
     ... 
0    6039
1    6039
2    6039
3    6039
4    6039
Name: user_id, Length: 836483, dtype: int64

In [36]:
ratings.movie_id

0      0
1      1
2      2
3      3
4      4
    ... 
0     40
1    513
2    160
3    176
4      4
Name: movie_id, Length: 836483, dtype: int64

In [37]:
ratings['user_id'].nunique()

6040

In [38]:
ratings['movie_id'].nunique()

3628

# 4) CSR matrix를 직접 만들어 봅시다.

In [39]:
import numpy as np
a = np.matrix([[0,1,3,5],
        [6,2,0,11],
        [0,7,9,0],
        [0,0,10,12]
       ])
a

matrix([[ 0,  1,  3,  5],
        [ 6,  2,  0, 11],
        [ 0,  7,  9,  0],
        [ 0,  0, 10, 12]])

In [65]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

# 노드와 다르게 ratings.count를 넣으면 len() of unsized object 오류가 뜸. 따라서 ratings['count']로 series로 입력
csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)), shape = (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

In [66]:
print(ratings['count'])

0    5
1    3
2    3
3    4
4    5
    ..
0    5
1    5
2    5
3    5
4    5
Name: count, Length: 836483, dtype: int64


# 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

 1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지
 2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
 3. use_gpu : GPU를 사용할 것인지
 4. iterations : epochs

In [42]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [43]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=200, regularization=0.01, use_gpu=False, iterations=20, dtype=np.float32)

In [44]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [45]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/20 [00:00<?, ?it/s]

# 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

### 내가 선호하는 영화

In [46]:
zimin, toy = user_to_idx['zimin'], movie_to_idx[1]
zimin_vector, toy_vector = als_model.user_factors[zimin], als_model.item_factors[toy]

print('슝=3')

슝=3


In [47]:
# zimin_vector

In [48]:
# toy_vector

In [49]:
# zimin과 toy story를 내적하는 코드
# factors와 iterations값을 수정헤가며 학습
# factors : 100, iterations : 10 / 60
# factors : 50,  iterations : 10 / 52
# factors : 150, iterations : 10 / 68
# factors : 200, iterations : 10 / 74
# factors : 250, iterations : 10 / 85
# factors : 200, iterations : 20 / 77

np.dot(zimin_vector, toy_vector)

0.7833734

### 내가 선호하지 않는 영화

In [50]:
heat = movie_to_idx[6]
heat_vector = als_model.item_factors[heat]
np.dot(zimin_vector, heat_vector)

0.04226034

# 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [51]:
favorite_movie = 126#선호 영화를 저장해서
movie_ids = movie_to_idx[favorite_movie]#인코딩하고
similar_movie = als_model.similar_items(movie_ids, N=15)#유사한 영화 추출(15개)
similar_movie

[(2074, 0.99999994),
 (2834, 0.8243777),
 (3021, 0.8013024),
 (2021, 0.8006244),
 (3617, 0.7988223),
 (3386, 0.7959823),
 (3170, 0.79505944),
 (2142, 0.7949743),
 (3570, 0.79366994),
 (3486, 0.793528),
 (3590, 0.79203224),
 (3375, 0.79170793),
 (3524, 0.7912791),
 (2020, 0.7909459),
 (3493, 0.79060864)]

In [52]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

[126,
 1822,
 243,
 634,
 1843,
 2563,
 734,
 1322,
 1107,
 3057,
 3164,
 3419,
 847,
 631,
 3595]

# 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [53]:
# movie_id를 title로 변경하여 딕셔너리에 저장
movie_to_title = {}
for i, k in zip(movies['movie_id'], movies['title']):
    movie_to_title[i] = k
movie_to_title[1]
    
def get_similar_movie(movie_id: int):
    movie_id = movie_to_idx[movie_id]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    similar_movie = [movie_to_title[i] for i in similar_movie]
    return similar_movie

print("슝=3")

슝=3


### toy story (1995)와 유사한 영화

In [54]:
get_similar_movie(1)

['toy story (1995)',
 'toy story 2 (1999)',
 'aladdin (1992)',
 'lion king, the (1994)',
 "bug's life, a (1998)",
 'groundhog day (1993)',
 'story of xinghua, the (1993)',
 'nobody loves me (keiner liebt mich) (1994)',
 'paris, france (1993)',
 'splendor (1999)']

### heat(action)와 유사한 영화

In [55]:
get_similar_movie(6) # heat (1995, action)

['heat (1995)',
 'ronin (1998)',
 'true romance (1993)',
 'in the line of fire (1993)',
 'out of sight (1998)',
 'seven (se7en) (1995)',
 "carlito's way (1993)",
 'long kiss goodnight, the (1996)',
 'mixed nuts (1994)',
 'out to sea (1997)']

### 유저의 취향과 유사한 영화 top 20

In [56]:
user = user_to_idx['zimin']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(50, 0.6228306),
 (173, 0.25642353),
 (33, 0.21621361),
 (458, 0.2127477),
 (1130, 0.21045566),
 (110, 0.19612263),
 (119, 0.18526527),
 (596, 0.18390682),
 (30, 0.18250354),
 (32, 0.17124534),
 (58, 0.15744473),
 (545, 0.15272406),
 (64, 0.15234669),
 (475, 0.15159908),
 (116, 0.14654353),
 (1736, 0.14607614),
 (330, 0.14262879),
 (10, 0.13900241),
 (663, 0.13730451),
 (435, 0.13708647)]

In [57]:
### 유저의 취향과 유사한 영화 top 20 제목

In [58]:
recommended = [idx_to_movie[i[0]] for i in movie_recommended]
recommended

[3114,
 653,
 588,
 367,
 60,
 1265,
 736,
 3489,
 2294,
 1566,
 648,
 317,
 1210,
 2302,
 590,
 1920,
 364,
 595,
 899,
 2023]

In [59]:
for i, k in enumerate(recommended):
    print('{} : {}'.format(i+1, movie_to_title[k]))

1 : toy story 2 (1999)
2 : dragonheart (1996)
3 : aladdin (1992)
4 : mask, the (1994)
5 : indian in the cupboard, the (1995)
6 : groundhog day (1993)
7 : twister (1996)
8 : hook (1991)
9 : antz (1998)
10 : hercules (1997)
11 : mission: impossible (1996)
12 : santa clause, the (1994)
13 : star wars: episode vi - return of the jedi (1983)
14 : my cousin vinny (1992)
15 : dances with wolves (1990)
16 : small soldiers (1998)
17 : lion king, the (1994)
18 : beauty and the beast (1991)
19 : singin' in the rain (1952)
20 : godfather: part iii, the (1990)


### 영향을 준 영화와 기여도

In [60]:
toystory2 = movie_to_idx[3114]

In [61]:
explain = als_model.explain(user, csr_data, itemid=toystory2)

In [62]:
[(movie_to_title[i[0]], i[1]) for i in explain[1]]

[('cry, the beloved country (1995)', 0.3589728082431962),
 ('waiting to exhale (1995)', 0.21490745363567876),
 ('radioland murders (1994)', 0.03459507564092045),
 ('congo (1995)', 0.01080306465744144),
 ('living in oblivion (1995)', -0.008446357416171954)]

# 루브릭 항목

### 1. CSR matrix가 정상적으로 만들어졌다.
사용자와 아이템 개수를 바탕으로 6040x3628의 정확한 사이즈로 만들었습니다.


### 2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.
사용자와 아이템 벡터 내적수치가 의미있게 형성되었습니다. 하이퍼 파라미터를 조정하여 보다 높은 내적 수치를 찾았습니다.


### 3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.
MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었습니다. 선호 영화를 입력하면 그와 유사한 영화를 정상적으로 추천하도록 구현하였습니다.

# 회고

이번 노드의 과정은 개념적인 설명이 주가 되어 이해하는 것이 다소 어려웠습니다. 특히 CSR matrix로의 변환을 이해하는 것이 다소 어려웠습니다. 그러나 결국 개념적으로 접근하여 왜 해당 형태로 변형해야 하는지를 이해하고 변환을 하는데 성공했습니다. 

이외에는 크고 작은 코드문제들이 있었습니다. 데이터의 형태부터 노드의 내용과 다소 다른 점이 있어 코드를 그대로 사용하려했다가 시간이 더 소요되었습니다. 결국 주어진 데이터를 원하는 형태로 변환하는 기능(영화 제목을 인덱스로, 또는 인덱스를 영화 제목으로 바꾸거나 movie_id를 영화 제목으로 바꾸는 등)을 구현하여 해결할 수 있었습니다.

하이퍼 파라미터의 경우 다음과 같은 여러 시도를 해보았습니다. 
* factors : 100, iterations : 10 / 60
* factors : 50,  iterations : 10 / 52
* factors : 150, iterations : 10 / 68
* factors : 200, iterations : 10 / 74
* factors : 250, iterations : 10 / 85
* factors : 200, iterations : 20 / 77


벡터를 몇차원으로 표현할 것인가에 대한 하이퍼 파라미터인 factors의 경우 커질수록 내적값이 증가하는 경향을 보였습니다. epoch에 해당하는 iterations의 경우 값이 커질수록 마찬가지로 내적값이 증가하였지만 10에서 20으로 증가시켜도 큰 상승을 보이지 않았습니다(소수점단위에 불과). factors가 커질수록 내적값이 증가하는 경향을 보였지만 factors의 값은 과적합을 방지하기 위해 일반적으로 50~200 사이의 값을 갖는다고 하여 200까지만 설정하였습니다. 따라서 최종적으로 factors : 200, iterations : 20으로 지정하여 학습하였습니다.

다만 마지막 영향을 얼마나 줬는가에 대한 해석이 다소 모호했습니다. 추천영화까지는 직관적으로 이해가 가능했지만 이 부분에서는 수치가 1에 가까워야 하는가? 또는 과연 해당 영화들이 정말 영향을 준 것인가?가 모호했습니다.

최근 가장 핫한 분야중 하나인 추천시스템에 대한 개념과 여러 방식에 대해 알 수 있어서 많은 도움이 되는 노드였습니다. 연관성을 단순히 주어진 feature만으로 비교하는 것이 아니고 '사용자가 아이템을 선택 했는가'의 정보를 벡터의 차원으로 나눠 특징을 비교한다(맞나..??)는 것을 알 수 있었습니다. 또 결과를 단순히 수치로 비교하는 것이 아닌 연구자의 인사이트를 통해 여러 의미를 도출 할수 있다는 점이 흥미로웠습니다. 다양한 추천 알고리즘이나 개념에 대해 더 공부하면 재밌을것 같습니다.